# Reference 

1. open dart official document
https://dart-fss.readthedocs.io/en/latest/welcome.html

2. 코딩 유치원 
https://coding-kindergarten.tistory.com/85

In [1]:
import pandas as pd
from tqdm import tqdm
import time

In [2]:
!pip install dart_fss

  Using cached pandas-1.2.5-cp39-cp39-win_amd64.whl (9.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.4.2
    Uninstalling pandas-1.4.2:
      Successfully uninstalled pandas-1.4.2


ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\user\\anaconda3\\envs\\codingNJ\\Lib\\site-packages\\~-ndas\\_libs\\algos.cp39-win_amd64.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
import dart_fss as dart

# 1. 상장기업 목록 다운

dart 서비스에 올라가 있는 기업들은 비상장 기업들의 재무정보도 올라와 있다. 
우리는 상장사의 재무정보만 보고 싶은 것이기 때문에, 전체 corp_list 중에서 상장기업들만 추출해야 한다.   

상장사 리스트를 얻기 위해 한국거래소 > 기본 통계 > 주식 > 종목정보 > 전종목 기본정보에서 전종목 기본정보를 다운받는다.   
http://data.krx.co.kr/contents/MDC/MDI/mdiLoader/index.cmd?menuId=MDC0201040201

In [4]:
# 다운받은 전종목 기본정보 불러오기 
all_stocks = pd.read_excel('./data_5655_20220311.xlsx')

C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\openpyxl\styles\stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


dart 패키지에서는 주식종목코드를 사용하는 것이아니라, dart 자체적인 corp_code(기업코드)를 가지고 정보를 불러온다.  
따라서 한국거래소에서 얻은 상장기업 주식코드를 가지고, dart의 데이터에서 상장기업 리스트와, 그들의 corp_code를 추출한다. 
여기서 주의 할 것은, 한국거래소에서 얻은 상장기업 주식코드에는 우선주도 포함된다는 점이다. 상장기업 주식코드중 보통주만 사용해야한다. 

In [5]:
# 상장기업 주식 코드 중 보통주만 걸러내기 
listed_companies = all_stocks[all_stocks['주식종류']=='보통주']

In [6]:
# DART API KEY 설정
api_key='7db60698e63bd621a006025500fe0436ee65cb47'
dart.set_api_key(api_key=api_key)

# DART 에 공시된 회사 리스트 불러오기
corp_list = dart.corp.get_corp_list()

# 상장 기업들의 기업코드와 회사 이름
listed_companies_corp_code = []
listed_companies_name = []

for stock_code in listed_companies['단축코드'] : 
    #corp_list.find_by_corp_name( ) 함수의 결과 데이터는 [[종목 코드]종목 이름]의 형태
    #이를 벗겨내기 위해서 corp_list.find_by_corp_name( )[0]으로 코딩함 
    try : 
        listed_companies_corp_code.append(corp_list.find_by_stock_code(stock_code).corp_code)
        listed_companies_name.append(corp_list.find_by_stock_code(stock_code).corp_name)
        
    except : 
        # corp_list
        print(stock_code)
        
corp_dic = { name:value for name, value in zip(listed_companies_corp_code, listed_companies_name) }

Output()

Output()

Output()

041140
220630
225850
329560
052190
039670
225330
050320
950180
138250
225860
270020
240340
330990
153360
066110
053660


corp_dic이라는, 딕셔너리가 만들어졌다.  
* key : 기업코드(주식코드가 아닌, dart에서 사용하는 자체 기업 코드)
* value : 기업명  
길이는 2482

In [7]:
len(corp_dic)

2482

In [8]:
from dart_fss import get_corp_list

# 모든 상장된 기업 리스트 불러오기
crp_list = get_corp_list()


# 다트에서 사용하는 회사코드를 이용한 찾기
crp_list.find_by_corp_code('00126380').product

'IMT2000 서비스용 동기식 기지국,교환국장비,데이터단말기,동영상휴대폰,핵심칩,반도체제품,사무,계산 및 회계용기계'

# 2. 재무제표 추출(금융업 제외, Opendart reader사용) 

재무제표를 추출하기 위해, dart open API를 쉽게 이용 할 수 있는 파이썬 패키지인 opendart reader를 사용한다.  
opeandart reader가 dart-fss 라이브러리 보다 더 빠르게 추출 가능하지만, 금융업은 추출하지 못한다는 단점이 있다.  
빠른 속도를 위해 Step2에서는 OpenDartReqder로 기업 정보를 추출 후, 추출하지 못한 기업들에 대해서 Step3에서 dart-fss로 재무정보를 추출한다. 

In [9]:
!pip install opendartreader
import OpenDartReader
dart2 = OpenDartReader(api_key)

### 표준화 함수
column_standardize 함수는 입력받은 대응표를 바탕으로 기업 계정과목을 표준화시킨다.  
이 함수는 먼저 'ifrs 코드'(국제공통회계코드) 를 바탕으로 필요한 정보를 추출한다.  
만약 기업이 ifrs코드를 사용하지 않고 임의의 계정과목명을 사용하여 추출에 실패할경우,   
대응표에서 오른쪽으로 한칸씩 이동하면서 ifrs코드에 대응되는 한글 계정과목명을 바꿔가며 정보 추출을 시도한다. 

input으로 대응표의 주소와, 표준화시켜야 할 데이터프레임을 받는다. 

In [10]:
def column_standardize(file_address, df ) :  
    # file adress는 대응표 주소 
    계정과목대응표 = pd.read_excel(file_address, index_col = 0)
    
    # 빈 데이터 프레임 생성 
    blank = pd.DataFrame(columns = list(계정과목대응표['기준']))
        
    for order, code in enumerate(list(계정과목대응표['ifrs_code'])) : 
        # 대응표의 ifrs코드로 필요한 정보를 추출한다. 
        try :
            blank.iloc[:,order] = df[code] 
            
        # 만약 실패할 경우, ifrs코드와 대응하는 한글 계정과목으로 추출을 시도한다. 
        except : 
            
            # 
            for alternative in list(계정과목대응표.loc[str(code),:]) : 

                try : 
                    # 주소변환 
                    blank.iloc[:,order] = df.iloc[:,list(df.isin([alternative]).any()).index(True)]
                    # print(f"대체 성공! {code}를 {alternative}로")
                    
                    break

                except :
                    #print(f"대체 실패! {code}를 {alternative}로"
                    pass  ## pass로 인해 혹시 라도 매칭이 안되는 부분을 blank null 처리하여 건너뛰게 된다. 
                
    # 아래코드는 ifrs-code, 한글 계정과목명을 제외한 숫자만을 인덱스로 추출하여 blank에 넣는 함수이다. 
    # 아래 코드를 주석 처리하면 결과가 확연히 달라지는 것을 볼 수 있음. 
    blank = blank.iloc[2,:]
    
    return blank

|### 에러로그   
데이터 수집시 수집이 안되는 기업들이 있다. 그런 기업들은 error_log 함수를 이용하여 csv에 저장한다.   
error_log 함수는 인풋으로 기업코드, corp_dic, report_code를 인풋으로 받아, 
어떤기업의 어떤 report를 추출할때 에러가 발생했는지 csv파일에 기록한다.   
함수를 실행할 때마다 새로 쓰는 것이 아니라, 기록돼 있는 곳에 append 하는 방식이다.   
아웃풋으로 활성화된 디렉토리에 error log.csv 파일이 생성된다. 

In [11]:
import csv

def error_log(corp_code, corp_dic, report_code) : 
    # corp_dic은 corp_code가 Key, corp_name이 value인 딕셔너리. 
    with open('error log.csv', 'a', newline='') as file  : 
        header = ["에러발생기업코드", "기업이름", "report_type"]
        writer = csv.DictWriter(file, fieldnames=header)
        #writer.writeheader()
        writer.writerow({"에러발생기업코드": corp_code, "기업이름": corp_dic[corp_code], "report_type": report_code})   
    
    return 

### 회계분기치환표 
opendart_reader 라이브러리에서 사용하는 report_code 를 이해하기 쉽도록 Q1, Q2 .. 로 치환하는 딕셔너리

In [1]:
회계분기치환표 = {
    '11013' : 'Q1', '11012' : 'Q2', '11014' : 'Q3', '11011' : 'Q4'
}

### 추출

In [13]:
계정과목대응표 = pd.read_excel('대응표.xlsx', index_col = 0)
    
# 빈 데이터 프레임 생성 
cleansed_finstats = pd.DataFrame(columns = list(계정과목대응표['기준']))


# 추출하고자 하는 재무정보의 연도 지정 
bsns_year = 2022
# 추출하고자 하는 분기(report_code) 지정 ('11013'=1분기보고서, '11012'=반기보고서, '11014'=3분기보고서, '11011'=사업보고서)
report_code = '11013'


for corp_code in tqdm(listed_companies_corp_code) : 

    # 만약 1년 전체를 획득할 생각이라면, for문을 돌린다. 
    #for report_code in ['11011','11014','11012','11013'] : 

    try : 
        # API로 부터 원본 재무데이터를 갖고 온다. 
        original = dart2.finstate_all(corp = corp_code , bsns_year = bsns_year, reprt_code = report_code)
        time.sleep(3)
        # 연도추출
        year = original.loc[0,'bsns_year'] 
        # 보고서타입 추출 
        reprt_type = original.loc[0,'thstrm_nm']
        # 자본변동표 제거 
        original = original.drop(original[original['sj_div']=='SCE'].index)
        original = original.T.loc[['account_id','account_nm','thstrm_amount'] ,:]
        original = original.reset_index()
        original.columns = original.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 

    except : 
        # 만약 원본 재무데이터를 갖고 오는데 실패할 경우, 로그로 기록한다. 
        error_log(corp_code = corp_code, corp_dic=corp_dic, report_code = report_code)
        #break
        continue

    #try : 
    # 칼럼 표준화 
    standardized_df = column_standardize(file_address = '대응표.xlsx', df = original)
    # 비어있는 회계연도와 회계분기를 기입한다. 
    standardized_df['회계연도'] = year
    standardized_df['회계분기'] = 회계분기치환표[report_code]

    # 기업 일반정보 추가. opendart reader 라이브러리 활용  
    company_info1 = dart2.company(corp_code)
    time.sleep(3)
    standardized_df['현지언어기업명'] = company_info1['corp_name']
    standardized_df['영문기업명'] = company_info1['corp_name_eng']
    standardized_df['통화구분코드'] = 'KRW'
    standardized_df['사업자등록번호'] = company_info1['bizr_no']
    standardized_df['법인등록번호'] = company_info1['jurir_no']
    standardized_df['설립일자'] = company_info1['est_dt']
    standardized_df['주식시장코드'] = company_info1['corp_cls']
    standardized_df['상장코드'] = company_info1['stock_code']
    standardized_df['현지언어기업주소'] = company_info1['adres']
    standardized_df['현지언어대표이사명'] = company_info1['ceo_nm']
    standardized_df['기업홈페이지URL'] = company_info1['hm_url']
    standardized_df['기업대표전화번호'] = company_info1['phn_no']
    standardized_df['대표팩스번호'] = company_info1['fax_no']

    # 기업 일반정보 추가. dart-fss 라이브러리 활용  
    standardized_df['현지언어산업군명'] = crp_list.find_by_corp_code(corp_code).sector
    time.sleep(3)
    standardized_df['현지언어주요제품명내용'] = crp_list.find_by_corp_code(corp_code).product
    

    # 분기가 바뀔 때마다 append. 
    cleansed_finstats = cleansed_finstats.append(standardized_df)


# 추출이 끝나면 저장 
cleansed_finstats.to_excel('6월2일최종.xlsx')


100%|████████████████████████████████████████████████████████████████████████████| 2482/2482 [5:16:18<00:00,  7.65s/it]


This Error occurs as a result of python script trying to connect to IBM service even before your wifi or ethernet connection is established. Have a try/catch to rectify or if trying to run as service then run service after network is established.

# 3. 재무제표 추출(금융업종 및 API로 추출 안되는 기업, Dart-fss 사용) 

## dart - fss 라이브러리 이용
dart-fss 라이브러리는 보아 하니 xbrl을 이용하는 것이 아닌, html에 직접 들어가서 정보를 찾아오는 방법이다.

In [22]:
# 삼전
fs = dart.fs.extract(corp_code= '00126380', bgn_de='20200101', report_tp = 'quarter', fs_tp=('bs', 'is', 'cis', 'cf'), separate = True)

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/5 [00:00<?, ?report/s]

In [23]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
bs_df = fs.show('bs', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
bs_df.drop(['level_1'],axis=1, inplace=True)
bs_df.columns = bs_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
bs_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
bs_df.rename(columns={bs_df.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
is_df = fs.show('cis', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
is_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
is_df.columns = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
is_df.rename(columns={is_df.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
cf_df = fs.show('cf', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
cf_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
cf_df.columns = cf_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
cf_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
cf_df.rename(columns={cf_df.columns[0]:'Account_year'}, inplace = True) 

In [27]:
is_df

,Account_year,ifrs-full_ProfitLoss,ifrs-full_OtherComprehensiveIncome,dart_OtherComprehensiveIncomeThatWillNotBeReclassifiedToProfitOrLossNetOfTax,entity00126380_udf_IS_2018510231444795_OtherComprehensiveIncomeThatWillNotBeReclassifiedToProfitOrLossNetOfTax,entity00126380_udf_IS_201710211163597_OtherComprehensiveIncomeThatWillNotBeReclassifiedToProfitOrLossNetOfTax,dart_OtherComprehensiveIncomeThatWillBeReclassifiedToProfitOrLossNetOfTax,ifrs-full_ComprehensiveIncome
1,[D410005] Statement of comprehensive income - ...,당기순이익(손실),기타포괄손익,후속적으로 당기손익으로 재분류되지 않는 포괄손익,기타포괄손익-공정가치금융자산평가손익,순확정급여부채(자산) 재측정요소,후속적으로 당기손익으로 재분류되는 포괄손익,총포괄손익
2,[D410005] Statement of comprehensive income - ...,Profit (loss) for the period,Other comprehensive income (loss),Items that not to be reclassified to profit or...,Gain (loss) on valuation of financial assets a...,"Remeasurement of net defined benefit assets, n...",Items that may be reclassified to profit or lo...,Total comprehensive income
3,20220101-20220331,"7,651,943,000,000.0","-116,928,000,000.0","-116,928,000,000.0","-53,901,000,000.0","-63,027,000,000.0",0.0,"7,535,015,000,000.0"
4,20210701-20210930,"8,594,666,000,000.0","-105,283,000,000.0","-105,283,000,000.0","-87,069,000,000.0","-18,214,000,000.0",NaN,"8,489,383,000,000.0"
5,20210401-20210630,"9,129,190,000,000.0","-80,229,000,000.0","-80,229,000,000.0","-61,578,000,000.0","-18,651,000,000.0",0.0,"9,048,961,000,000.0"
6,20210101-20211231,"30,970,954,000,000.0","-613,225,000,000.0","-613,225,000,000.0","-99,916,000,000.0","-513,309,000,000.0",0.0,"30,357,729,000,000.0"
7,20210101-20210930,"22,271,460,000,000.0","-169,506,000,000.0","-169,506,000,000.0","-94,671,000,000.0","-74,835,000,000.0",NaN,"22,101,954,000,000.0"
8,20210101-20210630,"13,676,794,000,000.0","-64,223,000,000.0","-64,223,000,000.0","-7,602,000,000.0","-56,621,000,000.0",0.0,"13,612,571,000,000.0"
9,20210101-20210331,"4,547,604,000,000.0","16,006,000,000.0","16,006,000,000.0","53,976,000,000.0","-37,970,000,000.0",0.0,"4,563,610,000,000.0"
10,20200701-20200930,"5,270,095,000,000.0","-28,298,000,000.0","-28,298,000,000.0","-18,268,000,000.0","-10,030,000,000.0",NaN,"5,241,797,000,000.0"


In [28]:
fs

{'bgn_de': '20200101',
 'corp_code': '00126380',
 'end_de': None,
 'financial statement': [{'title': '[D210005] Statement of financial position, '
                                   'current/non-current - Separate financial '
                                   'statements (Unit: KRW)'},
                         {'title': '[D310005] Income statement, by function of '
                                   'expense - Separate financial statements '
                                   '(Unit: KRW)'},
                         {'title': '[D410005] Statement of comprehensive '
                                   'income - Separate financial statements '
                                   '(Unit: KRW)'},
                         {'title': '[D520005] Statement of cash flows, '
                                   'indirect method - Separate financial '
                                   'statements (Unit: KRW)'}],
 'lang': 'ko',
 'report_tp': 'quarter',
 'separate': True,
 'separator': True}

In [87]:
# 상성화재손해보험, quarter, separate = False(default)
fs2 = dart.fs.extract(corp_code= '00139214', bgn_de='20200101', report_tp = 'quarter', fs_tp=('bs', 'is', 'cis', 'cf'), separate = True)

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/5 [00:00<?, ?report/s]

In [88]:
bs_df2 = fs2.show('bs', show_class = False, show_concept = True).T.reset_index()
bs_df2

,level_0,level_1,0,1,2,3,4,5,6,7,...,27,28,29,30,31,32,33,34,35,36
0,Statement of financial position(Unit: KWR),label_ko,자산,"I.현금및예치금(주석4,5,6,7,28)",II.금융자산,"1.당기손익인식금융자산(주석4,5,8)","2.매도가능금융자산(주석4,5,7,9,28)","3.만기보유금융자산(주석4,5,10)","4.대출채권(주석4,5,11,13,43)","5.기타수취채권(주석4,5,12,13,28,43)",...,특별계정부채(주석44)VIII.,부채총계,본자,I.자본금(주석29),II.자본잉여금(주석30),III.자본조정(주석31),IV.기타포괄손익누계액(주석32),익잉여금(주석33),자본총계,부채와자본총계
1,20220331,"(별도재무제표,)",,"854,395,161,054.0","72,567,397,103,598.0","1,663,784,158,556.0","43,132,356,102,625.0","134,861,917,780.0","26,246,987,261,602.0","1,389,407,663,035.0",...,"12,343,054,642,358.0","77,829,070,786,526.0",None,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","3,583,401,793,919.0","9,972,912,642,748.0","13,038,517,517,218.0","90,867,588,303,744.0"
2,20211231,"(별도재무제표,)",,"1,345,621,453,025.0","75,819,299,534,412.0","858,051,904,674.0","46,826,111,297,783.0","134,713,819,995.0","26,698,135,711,041.0","1,302,286,800,919.0",...,"12,368,448,009,828.0","77,984,585,785,362.0",,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","5,844,064,626,233.0","10,073,959,233,469.0","15,400,226,940,253.0","93,384,812,725,615.0"
3,20210930,"(별도재무제표,)",,"918,701,800,486.0","74,652,053,432,518.0","924,320,631,219.0","46,332,829,071,230.0","134,555,301,484.0","25,893,424,712,681.0","1,366,923,715,904.0",...,"10,984,638,695,908.0","76,540,532,581,940.0",None,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","5,739,835,908,917.0","10,003,607,146,475.0","15,225,646,135,943.0","91,766,178,717,883.0"
4,20210630,"(별도재무제표,)",,"998,828,961,477.0","74,382,203,235,381.0","1,186,798,766,885.0","46,453,028,365,123.0","134,400,393,532.0","25,405,903,340,156.0","1,202,072,369,685.0",...,"10,982,502,602,829.0","75,698,409,964,749.0",None,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","6,386,596,539,216.0","9,725,458,575,358.0","15,594,258,195,125.0","91,292,668,159,874.0"
5,20210331,"(별도재무제표,)",,"853,686,709,124.0","74,310,559,483,853.0","1,184,806,828,771.0","46,608,110,481,822.0","134,247,453,947.0","24,937,505,746,051.0","1,445,888,973,262.0",...,"10,959,553,026,757.0","75,687,519,204,565.0",None,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","6,574,145,051,777.0","9,412,829,812,125.0","15,469,177,944,453.0","91,156,697,149,018.0"
6,20201231,"(별도재무제표,)",,"1,666,541,185,203.0","75,664,860,014,698.0","1,068,652,358,508.0","47,381,946,121,361.0","134,104,881,258.0","25,830,803,911,698.0","1,249,352,741,873.0",...,"11,178,450,522,902.0","75,483,206,264,181.0",,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","7,400,082,856,501.0","9,355,471,739,939.0","16,237,757,676,991.0","91,720,963,941,172.0"
7,20200930,"(별도재무제표,)",,"826,429,110,353.0","73,099,815,600,009.0","1,578,095,533,364.0","45,940,289,710,247.0","133,951,958,303.0","24,249,039,574,020.0","1,198,438,824,075.0",...,"8,658,330,679,424.0","72,470,904,734,165.0",None,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","6,400,919,340,198.0","9,217,567,686,272.0","15,100,690,107,021.0","87,571,594,841,186.0"
8,20200630,"(별도재무제표,)",,"1,112,232,332,468.0","71,544,816,459,840.0","1,807,015,345,838.0","44,536,786,710,780.0","133,802,593,631.0","23,911,743,221,921.0","1,155,468,587,670.0",...,"8,565,150,715,491.0","71,693,387,858,618.0",None,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","5,962,295,808,399.0","9,022,003,210,678.0","14,466,502,099,628.0","86,159,889,958,246.0"
9,20200331,"(별도재무제표,)",,"1,187,993,801,696.0","69,973,793,262,728.0","936,141,696,248.0","44,513,099,249,714.0","133,650,929,153.0","23,034,554,817,153.0","1,356,346,570,460.0",...,"8,591,681,824,507.0","71,150,427,993,737.0",None,"26,473,418,500.0","939,233,350,327.0","-1,483,503,688,276.0","5,307,958,501,969.0","8,752,686,750,118.0","13,542,848,332,638.0","84,693,276,326,375.0"


In [89]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
bs_df2 = fs2.show('bs', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
bs_df2.drop(['level_1'],axis=1, inplace=True)
bs_df2.columns = bs_df2.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
bs_df2.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
bs_df2.rename(columns={bs_df2.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
is_df2 = fs2.show('cis', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
is_df2.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
is_df2.columns = is_df2.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
is_df2.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
is_df2.rename(columns={is_df2.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
cf_df2 = fs2.show('cf', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
cf_df2.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
cf_df2.columns = cf_df2.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
cf_df2.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
cf_df2.rename(columns={cf_df2.columns[0]:'Account_year'}, inplace = True) 

In [84]:
pd.DataFrame(cf_df2.columns)

,0
0,Account_year
1,Ⅰ.영업활동으로인한현금흐름
2,익(손실)당기순이1.
3,2.조정사항
4,이자수익
...,...
70,임대보증금의상환
71,증감의Ⅳ.현금및현금성자산
72,Ⅴ.기초현금및현금성자산
73,Ⅵ.외화표시현금및현금성자산의환율변동효과


국민은행의 문제는 1. 분기 데이터가 뽑히지 않는다는 것과 2. 컬럼 이름이 제각각인것이다.  
분기 데이터를 다운 받는데 왜 보이지가 않는 것일까? 
그리고 is_df2와 is_df 차이는 fs.extract의 parameter에서 separate = False/ True를 구분하여 뽑아본 것이다. 
몇몇 기업의 경우 separte = False (default) 로 하게 되면 추출이 안되는 경우가 있다. '
예를 들어, 로지스몬의 경우

In [102]:
# 이베스트 투자증권, annual, separate = True
fs3 = dart.fs.extract(corp_code= '00120182', bgn_de='20200101', report_tp = 'quarter', fs_tp=('bs', 'is', 'cis', 'cf'))

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Semiannual reports:   0%|          | 0/2 [00:00<?, ?report/s]

Quarterly reports:   0%|          | 0/5 [00:00<?, ?report/s]

In [103]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
bs_df3 = fs3.show('bs', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
bs_df3.drop(['level_1'],axis=1, inplace=True)
bs_df3.columns = bs_df3.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
bs_df3.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
bs_df3.rename(columns={bs_df3.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
is_df3 = fs3.show('cis', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
is_df3.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
is_df3.columns = is_df3.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
is_df3.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
is_df3.rename(columns={is_df3.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
cf_df3 = fs3.show('cf', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
cf_df3.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
cf_df3.columns = cf_df3.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
cf_df3.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
cf_df3.rename(columns={cf_df2.columns[0]:'Account_year'}, inplace = True) 

In [104]:
bs_df3

,Account_year,자산,"<주석39,42>현금및현금성자산","당기손익-공정가치측정금융자산<주석4,8,9,10,11,12,41,42>","위험회피목적파생상품자산<주석5,9,10,41,42>","<주석6,9,10,11,12,42>기타포괄손익-공정가치측정금융자산","<주석7,8,9,10,12,41,42>상각후원가측정금융자산","<주석13,41>관계기업및공동기업투자","<주석12,14,41>유형자산","<주석12,15>투자부동산",...,부채총계,자본,지배주주지분<주석26>,자본금,기타불입자본,기타자본구성요소,,비지배주주지분,자본총계,부채와자본총계
1,20220331,,"1,888,147,000,000.0","33,807,777,000,000.0","270,000,000.0","5,340,358,000,000.0","19,797,161,000,000.0","431,367,000,000.0","164,025,000,000.0","52,808,000,000.0",...,"54,708,175,000,000.0",,"6,973,463,000,000.0","1,782,560,000,000.0","1,867,583,000,000.0","250,291,000,000.0","3,073,029,000,000.0","5,092,000,000.0","6,978,555,000,000.0","61,686,730,000,000.0"
2,20211231,,"1,348,746,000,000.0","34,593,665,000,000.0",,"5,143,796,000,000.0","16,637,296,000,000.0","435,949,000,000.0","161,340,000,000.0","54,294,000,000.0",...,"51,718,694,000,000.0",,"6,818,154,000,000.0","1,609,400,000,000.0","1,641,600,000,000.0","264,584,000,000.0","3,302,570,000,000.0","5,151,000,000.0","6,823,305,000,000.0","58,541,999,000,000.0"
3,20210930,,"2,385,156,000,000.0","35,485,813,000,000.0",,"4,653,677,000,000.0","17,972,835,000,000.0","315,539,000,000.0","162,133,000,000.0","52,135,000,000.0",...,"54,826,299,000,000.0",,"6,438,314,000,000.0","1,531,275,000,000.0","1,520,120,000,000.0","273,362,000,000.0","3,113,557,000,000.0","5,257,000,000.0","6,443,571,000,000.0","61,269,870,000,000.0"
4,20210630,,"1,550,328,000,000.0","36,813,851,000,000.0","2,372,000,000.0","4,429,700,000,000.0","16,428,553,000,000.0","305,530,000,000.0","155,419,000,000.0","55,071,000,000.0",...,"54,013,100,000,000.0",,"6,133,475,000,000.0","1,531,275,000,000.0","1,520,120,000,000.0","183,140,000,000.0","2,898,940,000,000.0","5,164,000,000.0","6,138,639,000,000.0","60,151,739,000,000.0"
5,20210331,,"1,697,053,000,000.0","37,886,719,000,000.0","2,156,000,000.0","5,420,490,000,000.0","16,314,787,000,000.0","307,586,000,000.0","139,920,000,000.0","56,546,000,000.0",...,"56,430,030,000,000.0",,"5,862,455,000,000.0","1,531,275,000,000.0","1,520,120,000,000.0","182,559,000,000.0","2,628,501,000,000.0","5,166,000,000.0","5,867,621,000,000.0","62,297,651,000,000.0"
6,20201231,,"1,212,647,000,000.0","38,964,992,000,000.0","4,503,000,000.0","5,423,596,000,000.0","16,200,741,000,000.0","310,244,000,000.0","139,621,000,000.0","53,036,000,000.0",...,"56,935,559,000,000.0",,"5,797,722,000,000.0","1,531,275,000,000.0","1,520,120,000,000.0","164,622,000,000.0","2,581,705,000,000.0","5,152,000,000.0","5,802,874,000,000.0","62,738,433,000,000.0"
7,20200930,,"1,178,437,000,000.0","37,791,755,000,000.0","1,129,000,000.0","4,697,582,000,000.0","14,288,149,000,000.0","251,561,000,000.0","124,890,000,000.0","53,978,000,000.0",...,"53,146,687,000,000.0",,"5,767,067,000,000.0","1,531,275,000,000.0","1,519,983,000,000.0","210,797,000,000.0","2,505,012,000,000.0","6,503,000,000.0","5,773,570,000,000.0","58,920,257,000,000.0"
8,20200630,,"1,338,627,000,000.0","38,034,082,000,000.0","274,000,000.0","4,570,962,000,000.0","15,729,042,000,000.0","257,264,000,000.0","125,031,000,000.0","55,213,000,000.0",...,"55,068,131,000,000.0",,"5,552,903,000,000.0","1,531,275,000,000.0","1,519,983,000,000.0","236,340,000,000.0","2,265,305,000,000.0","6,889,000,000.0","5,559,792,000,000.0","60,627,923,000,000.0"
9,20200331,,"1,293,156,000,000.0","35,237,025,000,000.0","375,000,000.0","4,540,257,000,000.0","16,174,231,000,000.0","165,803,000,000.0","123,200,000,000.0","55,587,000,000.0",...,"52,758,429,000,000.0",,"5,304,014,000,000.0","1,531,275,000,000.0","1,519,983,000,000.0","216,944,000,000.0","2,035,812,000,000.0","5,364,000,000.0","5,309,378,000,000.0","58,067,807,000,000.0"
10,20191231,,"996,123,000,000.0","32,663,997,000,000.0","2,207,000,000.0","4,135,531,000,000.0","11,617,742,000,000.0","149,430,000,000.0","126,205,000,000.0","55,621,000,000.0",...,"44,682,967,000,000.0",,"5,385,079,000,000.0","1,531,275,000,000.0","1,519,983,000,000.0","176,433,000,000.0","2,157,388,000,000.0","7,006,000,000

In [108]:
pd.DataFrame(cf_df3.columns)

,0
0,Statement of cash flows(Unit: KWR)
1,Ⅰ.영업활동으로인한현금흐름
2,1.법인세비용차감전순이익
3,2.손익조정항목<주석39>
4,3.자산ㆍ부채의변동<주석39>
5,4.이자수취액
6,5.이자지급액
7,6.배당금수취액
8,7.법인세납부액
9,Ⅱ.투자활동으로인한현금흐름


In [67]:
# 로지스몬, annual, separate = False
fs4 = dart.fs.extract(corp_code= '01060735', bgn_de='20200101', report_tp = 'annual', fs_tp=('bs', 'is', 'cis', 'cf'), separate = False)

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:1388: RuntimeWarning: Unable to extract financial statements: {'rcp_no': '20220323000792', 'corp_code': '00113562', 'corp_name': '롯데손해보험', 'stock_code': '000400', 'corp_cls': 'Y', 'report_nm': '사업보고서 (2021.12)', 'flr_nm': '롯데손해보험', 'rcept_dt': '20220323', 'rm': ''}.
  warnings.warn(warnings_text, RuntimeWarning)
C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:1388: RuntimeWarning: Unable to extract financial statements: {'rcp_no': '20210323001100', 'corp_code': '00113562', 'corp_name': '롯데손해보험', 'stock_code': '000400', 'corp_cls': 'Y', 'report_nm': '사업보고서 (2020.12)', 'flr_nm': '롯데손해보험', 'rcept_dt': '20210323', 'rm': ''}.
  warnings.warn(warnings_text, RuntimeWarning)
C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:1388: RuntimeWarning: Unable to extract financial statements: {'rcp_no': '20210129000010', 'corp_code': '00113562', 'corp_name': '롯데손해보

NotFoundConsolidated: ('Could not find consolidated financial statements', "An error occurred while fetching or analyzing {'rcp_no': '20210129000010', 'corp_code': '00113562', 'corp_name': '롯데손해보험', 'stock_code': '000400', 'corp_cls': 'Y', 'report_nm': '[기재정정]사업보고서 (2019.12)', 'flr_nm': '롯데손해보험', 'rcept_dt': '20210129', 'rm': ''}.")

In [246]:
# 로지스몬, quarter, separate = True
fs4 = dart.fs.extract(corp_code= '01060735', bgn_de='20200101', report_tp = 'quarter', fs_tp=('bs', 'is', 'cis', 'cf'), separate = True)

Annual reports:   0%|          | 0/3 [00:00<?, ?report/s]

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1411, in extract
    statements, label_df = merge_fs(statements, nstatements, fs_tp=fs_tp, label_df=label_df)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1019, in merge_fs
    ndata, nlabels = compare_func(column, df, ndf, label_df[tp], ndata, nlabels)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 98, in compare_df_and_ndf_cnn
    concept_id = guess_concept_id(label)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 40, in guess_concept_id
    return guess(text)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 30, in guess
    c = Classifier()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\utils\singleton.py", line 8, i

NoDataReceived: ('조회된 데이타가 없습니다.', "An error occurred while fetching or analyzing {'rcp_no': '20200330003081', 'corp_code': '01060735', 'corp_name': '로지스몬', 'stock_code': '223220', 'corp_cls': 'N', 'report_nm': '사업보고서 (2019.12)', 'flr_nm': '로지스몬', 'rcept_dt': '20200330', 'rm': ''}.")

로지스몬의 사례를 보면, separate = True 또는 False가 추출의 성공 여부를 다른다. 
또한, 로지스몬은 아예 분기데이터가 없어서  
seaparte = True로 두어도 reprt_tp = 'quarter' 일때 에러가 발생한다.   
(NoDataReceived : '조회된 데이타가 없습니다' 라는 에러)  
그렇다면, 데이터 다운로드가 되는데 추출이 안되는 국민은행 같은 경우는 무엇이 문제인가?

국민은행

In [248]:
# 국민은행, quarter, separate = True
fs5 = dart.fs.extract(corp_code= '00688996', bgn_de='20200101', report_tp = 'quarter', fs_tp=('bs', 'is', 'cis', 'cf'), separate = True)

Annual reports:   0%|          | 0/5 [00:00<?, ?report/s]

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1411, in extract
    statements, label_df = merge_fs(statements, nstatements, fs_tp=fs_tp, label_df=label_df)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1019, in merge_fs
    ndata, nlabels = compare_func(column, df, ndf, label_df[tp], ndata, nlabels)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 98, in compare_df_and_ndf_cnn
    concept_id = guess_concept_id(label)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 40, in guess_concept_id
    return guess(text)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 30, in guess
    c = Classifier()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\utils\singleton.py", line 8, i

  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 132, in __init__
    self.konlpy = Kkma()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\konlpy\tag\_kkma.py", line 44, in __init__
    jvm.init_jvm(jvmpath, max_heap_size)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\konlpy\jvm.py", line 55, in init_jvm
    jvmpath = jvmpath or jpype.getDefaultJVMPath()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\jpype\_jvmfinder.py", line 74, in getDefaultJVMPath
    return finder.get_jvm_path()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\jpype\_jvmfinder.py", line 212, in get_jvm_path
    raise JVMNotFoundException("No JVM shared library file ({0}) "
jpype._jvmfinder.JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.
C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:141

Semiannual reports:   0%|          | 0/4 [00:00<?, ?report/s]

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1411, in extract
    statements, label_df = merge_fs(statements, nstatements, fs_tp=fs_tp, label_df=label_df)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1019, in merge_fs
    ndata, nlabels = compare_func(column, df, ndf, label_df[tp], ndata, nlabels)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 98, in compare_df_and_ndf_cnn
    concept_id = guess_concept_id(label)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 40, in guess_concept_id
    return guess(text)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 30, in guess
    c = Classifier()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\utils\singleton.py", line 8, i

  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 132, in __init__
    self.konlpy = Kkma()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\konlpy\tag\_kkma.py", line 44, in __init__
    jvm.init_jvm(jvmpath, max_heap_size)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\konlpy\jvm.py", line 55, in init_jvm
    jvmpath = jvmpath or jpype.getDefaultJVMPath()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\jpype\_jvmfinder.py", line 74, in getDefaultJVMPath
    return finder.get_jvm_path()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\jpype\_jvmfinder.py", line 212, in get_jvm_path
    raise JVMNotFoundException("No JVM shared library file ({0}) "
jpype._jvmfinder.JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.
C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:141

Quarterly reports:   0%|          | 0/9 [00:00<?, ?report/s]

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1411, in extract
    statements, label_df = merge_fs(statements, nstatements, fs_tp=fs_tp, label_df=label_df)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1019, in merge_fs
    ndata, nlabels = compare_func(column, df, ndf, label_df[tp], ndata, nlabels)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 98, in compare_df_and_ndf_cnn
    concept_id = guess_concept_id(label)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 40, in guess_concept_id
    return guess(text)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 30, in guess
    c = Classifier()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\utils\singleton.py", line 8, i

  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 132, in __init__
    self.konlpy = Kkma()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\konlpy\tag\_kkma.py", line 44, in __init__
    jvm.init_jvm(jvmpath, max_heap_size)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\konlpy\jvm.py", line 55, in init_jvm
    jvmpath = jvmpath or jpype.getDefaultJVMPath()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\jpype\_jvmfinder.py", line 74, in getDefaultJVMPath
    return finder.get_jvm_path()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\jpype\_jvmfinder.py", line 212, in get_jvm_path
    raise JVMNotFoundException("No JVM shared library file ({0}) "
jpype._jvmfinder.JVMNotFoundException: No JVM shared library file (jvm.dll) found. Try setting up the JAVA_HOME environment variable properly.
C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py:141

Traceback (most recent call last):
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1411, in extract
    statements, label_df = merge_fs(statements, nstatements, fs_tp=fs_tp, label_df=label_df)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\fs\extract.py", line 1019, in merge_fs
    ndata, nlabels = compare_func(column, df, ndf, label_df[tp], ndata, nlabels)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 98, in compare_df_and_ndf_cnn
    concept_id = guess_concept_id(label)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\fs_search.py", line 40, in guess_concept_id
    return guess(text)
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss_classifier\classifier.py", line 30, in guess
    c = Classifier()
  File "C:\Users\user\anaconda3\envs\codingNJ\lib\site-packages\dart_fss\utils\singleton.py", line 8, i

In [251]:
fs5.show('bs', show_class = False, show_concept = True)

,Statement of financial position(Unit: KWR),20211231,20201231
,label_ko,"(별도재무제표,)","(별도재무제표,)"
0,자산,,
1,"(주석4,5,6,29)Ⅰ.현금및예치금","608,076,000,000.0","23,084,000,000.0"
2,"(주석4,5,7)Ⅱ.당기손익-공정가치측정금융자산","440,760,000,000.0","474,262,000,000.0"
3,"(주석4,5,8)Ⅲ.상각후원가측정대출채권","249,128,000,000.0","179,542,000,000.0"
4,(주석9)Ⅳ.종속기업투자,"26,741,438,000,000.0","26,519,880,000,000.0"
5,(주석10)Ⅴ.유형자산,"4,444,000,000.0","7,730,000,000.0"
6,(주석11)Ⅵ.무형자산,"16,673,000,000.0","13,267,000,000.0"
7,(주석17)Ⅶ.순확정급여자산,"221,000,000.0",
8,(주석13)Ⅷ.이연법인세자산,"5,583,000,000.0","3,189,000,000.0"


In [254]:
# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
bs_df = fs.show('bs', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
bs_df.drop(['level_1'],axis=1, inplace=True)
bs_df.columns = bs_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
bs_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
bs_df.rename(columns={bs_df.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
is_df = fs.show('cis', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
is_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
is_df.columns = is_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
is_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
is_df.rename(columns={is_df.columns[0]:'Account_year'}, inplace = True) 

# class 를 제거, ifrs 공식 용어인 concept 제거, 이후 transpose, reset_index 
cf_df = fs.show('cf', show_class = False, show_concept = True).T.reset_index()
# level_1은 쓸모없는 설명 열 이므로 제거 
cf_df.drop(['level_1'],axis=1, inplace=True)
# 이렇게 전처리된 데이터 프레임에서 1번째 인덱스는 각 회계 항목의 영어이름(label_en)이다. 
cf_df.columns = cf_df.iloc[0]  # 0번째 행을 열이름으로 바꾼다. 이렇게 함으로써 열 이름이 회계 세부 항목 이름으로 바뀌게 된다. 
cf_df.drop(0,inplace=True)  # 열이름으로 바꾼후, 0번째 행은 제거.
# 0번째 열이름이 이 재무제표의 title로 되어 있다. 이것을 account_year로 바꾼다. 
cf_df.rename(columns={cf_df.columns[0]:'Account_year'}, inplace = True) 

In [260]:
fs.save('test.xlsx')

'C:\\Users\\user\\codes\\2022 BDC\\fsdata\\test.xlsx'

In [255]:
is_df

,Account_year,(주석21)Ⅰ.순이자손익,1.이자수익,상각후원가측정금융상품이자수익,당기손익-공정가치측정금융상품이자수익,2.이자비용,(주석22)Ⅱ.순수수료손익,1.수수료수익,2.수수료비용,(주석23)Ⅲ.당기손익-공정가치측정금융상품순손익,...,X.법인세비용차감전순이익,(주석27)XI.법인세수익,XII.당기순이익,XIII.법인세비용차감후당기기타포괄손익,당기손익으로재분류되지않는포괄손익,1.순확정급여부채의재측정요소,XIV.당기총포괄이익,(주석28)XV.주당이익,기본주당이익,희석주당이익
1,20210101-20211231,"-111,077,000,000.0","9,392,000,000.0","6,548,000,000.0","2,844,000,000.0","-120,469,000,000.0","-8,157,000,000.0","975,000,000.0","-9,132,000,000.0","20,250,000,000.0",...,"1,436,585,000,000.0","2,281,000,000.0","1,438,866,000,000.0","-298,000,000.0","-298,000,000.0","-298,000,000.0","1,438,568,000,000.0",,"3,509.0","3,436.0"
2,20200101-20201231,"-124,393,000,000.0","8,044,000,000.0","3,788,000,000.0","4,256,000,000.0","-132,437,000,000.0","-8,338,000,000.0","841,000,000.0","-9,179,000,000.0","12,663,000,000.0",...,"1,379,366,000,000.0","49,000,000.0","1,379,415,000,000.0","-368,000,000.0","-368,000,000.0","-368,000,000.0","1,379,047,000,000.0",,"3,482.0","3,438.0"


In [259]:
cf_df

,Account_year,Ⅰ.영업활동으로부터의현금흐름,1.당기순이익,2.손익조정사항,(1)감가상각비및상각비,(2)신용손실충당금전입액,(3)주식보상비용,(4)순이자손익,(5)당기손익-공정가치측정금융자산평가손익,(6)외화환산손익,...,2.차입부채의상환,3.사채의발행,4.사채의상환,5.배당금의지급,6.리스부채의원금상환,7.신종자본증권의발행,8.신종자본증권배당금지급,Ⅳ.현금및현금성자산의순증감(I+II+III),Ⅴ.기초현금및현금성자산(주석29),Ⅵ.기말현금및현금성자산(주석29)
1,20210101-20211231,"1,346,081,000,000.0","1,438,866,000,000.0","18,509,000,000.0","6,506,000,000.0","417,000,000.0","9,230,000,000.0","4,379,000,000.0","-355,000,000.0",,...,"-100,000,000,000.0","389,405,000,000.0","-970,000,000,000.0","-981,879,000,000.0","-535,000,000.0","1,142,203,000,000.0","-71,538,000,000.0","494,992,000,000.0","23,081,000,000.0","518,073,000,000.0"
2,20200101-20201231,"1,389,345,000,000.0","1,379,415,000,000.0","13,888,000,000.0","4,357,000,000.0","465,000,000.0","4,034,000,000.0","3,705,000,000.0","2,606,000,000.0","-117,000,000.0",...,"-340,000,000,000.0","1,537,091,000,000.0","-940,000,000,000.0","-861,092,000,000.0","-610,000,000.0","1,296,693,000,000.0","-22,860,000,000.0","4,547,000,000.0","18,534,000,000.0","23,081,000,000.0"


In [45]:
import requests
payload = dict(rcpNo='20220516002493')
#if self.dcm_no:
#    payload['dcmNo'] = self.dcm_no
resp = requests.get(url= 'https://dart.fss.or.kr/dsaf001/main.do?rcpNo=20220516002493')
#self.html = BeautifulSoup(resp.text, 'html.parser')


In [51]:
import re
# Javascript was changed in newdart
regex_title = re.compile(r"node.*\'text\'.*\"(.+)\"")
regex_rcp_no = re.compile(r"node.*\'rcpNo\'.*\"(.+)\"")
regex_dcm_no = re.compile(r"node.*\'dcmNo\'.*\"(.+)\"")
regex_ele_id = re.compile(r"node.*\'eleId\'.*\"(.+)\"")
regex_offset = re.compile(r"node.*\'offset\'.*\"(.+)\"")
regex_length = re.compile(r"node.*\'length\'.*\"(.+)\"")
regex_dtd = re.compile(r"node.*\'dtd\'.*\"(.+)\"")
s = str(bs4)
titles = regex_title.findall(s)
rcp_nos = regex_rcp_no.findall(s)
dcm_nos = regex_dcm_no.findall(s)
ele_ids = regex_ele_id.findall(s)
offsets = regex_offset.findall(s)
lengths = regex_length.findall(s)
dtds = regex_dtd.findall(s)


In [55]:
titles
rcp_nos
dcm_nos
offsets

['663',
 '17240',
 '17635',
 '17725',
 '21404',
 '21806',
 '21994',
 '51788',
 '52167',
 '52502',
 '55168',
 '1202727',
 '1454726',
 '1622226',
 '1979163',
 '1979281',
 '1979771',
 '2163084',
 '3168169',
 '3286371',
 '3555260',
 '3567116',
 '4756244',
 '4799118',
 '4799344',
 '4799464',
 '4841182',
 '4841468',
 '4841572',
 '4842148',
 '4843160',
 '4843761',
 '4867001',
 '4867104',
 '4966579',
 '4967584',
 '4968488',
 '4999393',
 '4999534',
 '5000845',
 '5245456',
 '5245846',
 '5246857',
 '5246972',
 '5247383',
 '5247751',
 '5248122',
 '5287038',
 '5389658',
 '5400617',
 '5414275',
 '5443762',
 '5458073',
 '5469790',
 '5478485',
 '5509026',
 '5547616',
 '5794668',
 '5816219',
 '5816311',
 '5816750']

In [77]:
from dart_fss.filings import search as search_filings

In [83]:
all_pblntf_detail_ty = ('A001', 'A002', 'A003')
search_filings(corp_code='00688996', bgn_de='20200101', end_de='20220531', pblntf_detail_ty=all_pblntf_detail_ty[2], page_count=100, last_reprt_at='Y')

{'page_count': 100,
 'page_no': 1,
 'report_list': [{'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20220516',
                  'rcp_no': '20220516002493',
                  'report_nm': '분기보고서 (2022.03)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20211115',
                  'rcp_no': '20211115002116',
                  'report_nm': '분기보고서 (2021.09)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20210517',
                  'rcp_no': '20210517001542',
                  'report_nm': '분기보고서 (2021.03)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20210427',
                  'rcp_no': '20210427000513',
                  'report_nm': '[기재정정]분기보고서 (2020.09)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20210427',
                  'rcp_no': '20210427000485',
                  'report_nm': '[기재정정]분기보고서 (2020.03)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20210427',
                  'rcp_no': '20210427000463',
                  'report_nm': '[기재정정]분기보고서 (2019.09)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20210427',
                  'rcp_no': '20210427000437',
                  'report_nm': '[기재정정]분기보고서 (2019.03)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20210427',
                  'rcp_no': '20210427000402',
                  'report_nm': '[기재정정]분기보고서 (2018.09)',
                  'rm': '',
                  'stock_code': '105560'},
                 {'corp_cls': 'Y',
                  'corp_code': '00688996',
                  'corp_name': 'KB금융',
                  'flr_nm': 'KB금융',
                  'rcept_dt': '20210427',
                  'rcp_no': '20210427000369',
                  'report_nm': '[기재정정]분기보고서 (2018.03)',
                  'rm': '',
                  'stock_code': '105560'}],
 'total_count': 9,
 'total_page': 1}

In [140]:
from dart_fss.filings import reports

aReport = reports.Report(rcept_no = '20220516002493')

In [145]:
from dart_fss.utils import str_insert_whitespace as ws
inc = ['재무제표', '감사보고서']
exc = ['주석', '결합', '의견', '수정', '검토보고서']

includes = ' OR '.join([ws(x) for x in inc])
excludes = ' OR '.join([ws(x) for x in exc])

query = {
    'includes': includes,
    'excludes':excludes,
    'scope': ['attached_reports', 'pages'],
    'options': {'title': True}  # 첨부보고서 및 연결보고서의 title 까지 검색
}

query['excludes'] += ' OR ' + ws('연결')


from dart_fss.fs.extract import report_find_all

count, fs_table = report_find_all(report = aReport, query = query , fs_tp = ('bs', 'is', 'cis', 'cf') , separate=True)


In [149]:
aReport

{'rcp_no': '20220516002493', 'rpt_nm': '2022.05.16 분기보고서'}